In [59]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os
os.environ['OPENAI_API_KEY'] = ''


In [60]:
specification = { "specifications": "ballpoint pen, black, metal", "quantity": 30, "price": 5, "num_days": 7, "need_logo": 'false' }

In [67]:
def gpt_draft_mail(specification, vendor_name):
    # Create a GPT prompt
    pormpt = f"Write a mail to a vendor named '{vendor_name}' on behalf of Vishwa Mohan Singh (salutations), asking for a quotation for the following specifications:\nSpecifications: {specification['specifications']}\nQuantity: {specification['quantity']}\nOur Price: {specification['price']} Euros\nNumber of days Required: {specification['num_days']}\nNeed logo: {specification['need_logo']}\n\nMail:"

    mail_assistant = ChatOpenAI()
    messages = [
        SystemMessage(
            content="You are an AI assistant that is supposed to write a mail to the vendor asking for a quotation and time of delivery. Specify our required price and days as well."
        ),
        HumanMessage(content=pormpt),
    ]
    response = mail_assistant(messages)

    return response.content

In [68]:
print(gpt_draft_mail(specification, "ABC Corp."))

Dear ABC Corp.,

I hope this email finds you well. My name is Vishwa Mohan Singh, and I am reaching out to you on behalf of our company. We are interested in purchasing ballpoint pens with the following specifications: black ink, metal body.

We require a quantity of 30 pens. Could you please provide us with a quotation for this order? We have a budget of 5 Euros per pen.

Additionally, we would appreciate it if you could let us know the number of days required for the delivery. Our deadline is in 7 days, so timely delivery is crucial for us.

Please note that we do not require any logo on the pens.

Thank you in advance for your prompt attention to this matter. We look forward to receiving your quotation and delivery details.

Best regards,

Vishwa Mohan Singh


In [53]:
function_json = {
  "name": "get_vendor_quotation",
  "parameters": {
    "type": "object",
    "properties": {
      "requirements_satisfied": {
        "type": "boolean",
        "description": "Can the requirements be satisfied by the vendor? Make it true even if some requirements can be satisfied."
      },
      "unfullfiled_requirements": {
        "type": "string",
        "description": "What requirements cannot be satisfied by the vendor? Separate requirements with ';'. NA if there are no missing requirements."
      },
      "quotation": {
        "type": "number",
        "description": "Price per unit of the product. Only set to NA if the requirements cannot be satisfied at all. Otherwise ask the user for the exact price."
      },
      "procurement_days": {
        "type": "number",
        "description": "Number of days to fullfil the requirements. Only set to NA if the requirements cannot be satisfied at all. Otherwise ask the user for the days"
      }
    },
    "required": [
      "requirements_satisfied",
      "unfullfiled_requirements",
      "quotation",
      "procurement_days"
    ]
  },
  "description": "Check if the vendor can fullfil the requirements and get the vendor quotation and procurement days from the mail."
}

In [54]:
proc_assistant = OpenAIAssistantRunnable.create_assistant(
    name="Procurement_Bot_Vishwa",
    instructions=f"You are an AI assistant who is supposed to get the quotation and time of delivery from the vendor. These are the requirements:{specification}. You are supposed to collect the three main information: Can the requirements be met, what is the price and how long will it take to deliver (in days). All these requirements need to be provided by the vendor in chat. Do not invoke the function until you have all the information or if the vendor has rejected it. Always write your responses in form of a mail on behalf of Vishwa Singh.",
    tools=[{"type": "function", "function": function_json, "name": "get_specifications"}],
    model="gpt-4-1106-preview",)


# mail_assistant = OpenAIAssistantRunnable.create_assistant(
#     name="Mail Bot",
#     instructions="You are an AI assistant that is supposed to write a mail to the vendor asking for a quotation.",
#     tools=[],
#     model="gpt-4-1106-preview",
# )

In [57]:
output = proc_assistant.invoke({"content": "Yes, we can satisfy all the requirements, but we only have blue plastic pens."})

In [58]:
output

[RequiredActionFunctionToolCall(id='call_g0YAc8BxnqZVSRadRXyjzsTS', function=Function(arguments='{"requirements_satisfied":true,"unfullfiled_requirements":"color: not black; material: not metal","quotation":0,"procurement_days":0}', name='get_vendor_quotation'), type='function')]